In [9]:
import pandas as pd
from sklearn.metrics import make_scorer, f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import joblib

In [10]:
db1 = pd.read_csv('../data/db1.csv')
db2 = pd.read_csv('../data/db2.csv')

y_train = db1['Diabetes_binary'].replace({2: 1})
X_train = db1.drop('Diabetes_binary', axis=1)

X_test = db2.drop('Diabetes_binary', axis=1)
y_test = db2['Diabetes_binary']


In [ ]:
cv = joblib.load("../models/cv_res.pkl")
cv

array([0.82229581, 0.82848471, 0.82284768, 0.82698675, 0.82643488,
       0.82071902, 0.82304478, 0.82430621, 0.82075844, 0.8230842 ])

In [8]:
pipeline = joblib.load("../models/pipeline_side.pkl")
pipeline

,steps,"[('adasyn', ...), ('scaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,sampling_strategy,'auto'
,random_state,42
,n_neighbors,2
,copy,True
,with_mean,True
,with_std,True
,objective,'binary:logistic'


In [ ]:
# Check booster-level of XGB
md = XGBClassifier()
print(md.get_params().keys())

dict_keys(['objective', 'base_score', 'booster', 'callbacks', 'colsample_bylevel', 'colsample_bynode', 'colsample_bytree', 'device', 'early_stopping_rounds', 'enable_categorical', 'eval_metric', 'feature_types', 'feature_weights', 'gamma', 'grow_policy', 'importance_type', 'interaction_constraints', 'learning_rate', 'max_bin', 'max_cat_threshold', 'max_cat_to_onehot', 'max_delta_step', 'max_depth', 'max_leaves', 'min_child_weight', 'missing', 'monotone_constraints', 'multi_strategy', 'n_estimators', 'n_jobs', 'num_parallel_tree', 'random_state', 'reg_alpha', 'reg_lambda', 'sampling_method', 'scale_pos_weight', 'subsample', 'tree_method', 'validate_parameters', 'verbosity'])


In [12]:
par_grid = {
    'model__n_estiators': [200, 400],
    'model__max_depth' : [4,6,8],
    'model__learning_rate': [0.01, 0.02, 0.05, 0.1],
    'model__subsample': [0.7, 1.0],
    'model__colsample_bytree': [0.7, 1.0]
}

grid = GridSearchCV(
    estimator = pipeline,
    param_grid = par_grid,
    scoring = make_scorer(f1_score),
    cv = cv,
    verbose = 2,
    n_jobs = -1
)

grid.fit(X_train, y_train)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


TypeError: cannot unpack non-iterable numpy.float64 object

In [ ]:
print("Best params:", grid.best_params_)
print("Best score:", grid.best_score_)

best_model = grid.best_estimator_

joblib.dump(best_model, "models/xgb_stage2_best.pkl")